# Selecting A Text File

In [1]:
#using the Tkinter graphical user interface (GUI) to select text file
from tkinter import filedialog
from tkinter import *
import os

root = Tk()
cwd = os.getcwd() # getting current working directory
root.filename = filedialog.askopenfilename(initialdir = cwd,title = "Please select document",filetypes = (("all files","*.*"),("plain text","*.txt"),('pdf','*.pdf')))


# Extracting Text From The File

In [2]:
#extracting text from the file using Apache Tika
import tika
from tika import unpack
parsed = unpack.from_file(root.filename)
text=parsed["content"]   #obtaining the document's text


# Processing The Text Using SpaCy

In [3]:
import spacy
#loading the SpaCy 'small' model
nlp = spacy.load("en_core_web_sm")

#Processing the text
doc = nlp(text)

#obtaining entity information
ent_text=[]
ent_label=[]
ent_sentence=[]
for entity in doc.ents:
    ent_text.append(entity.text) #the entity's text
    ent_label.append(entity.label_) # the type of entity e.g. PERSON,GPE, etc
    ent_sentence.append(entity.sent.text.replace('\n',' ')) # some context of the entity

len(ent_label),len(ent_text),len(ent_sentence),len(doc.ents) # check number of entities + related info

(2774, 2774, 2774, 2774)

# Exporting Extracted Information To A File

In [4]:
#Saving the information as a dataframe
import pandas as pd
df=pd.DataFrame({'Entity': ent_text,'Type': ent_label,'Context':ent_sentence})
df.head()

,Entity,Type,Context
0,Latin,NORP,Science Science (from the Latin word scienti...
1,Egypt,GPE,The earliest roots of science can be traced to...
2,Mesopotamia,PRODUCT,The earliest roots of science can be traced to...
3,Greek,NORP,"Their contributions to mathematics, astronomy,..."
4,Empire,GPE,"After the fall of the Western Roman Empire, kn..."


In [5]:
#Separating info for each entity. This gives a list of (label, label_dataframe) tuples.
entities=list(df.groupby('Type'))

#Exporting the entity info into an excel file, with a sheet for each type:
save_dir=filedialog.askdirectory(title='Select destination directory') # a GUI to select destination directory
with pd.ExcelWriter(save_dir+'/text-mining.xlsx') as writer:
    for x in entities:
        x[1].to_excel(writer, sheet_name=x[0], index=False)
